# Example 0 - Working with a simple function

In this notebook we will give an overview of the API provided by `malcolm-appraiser` through a simple example.

The math necessary to follow this notebook is explained in [bayesian_inference.md](./bayesian_inference.md).

## Forward model

We are given a forward model in the form of the following function

$$f(x,y,z) := xyz + 2yz - 3xz + 2$$

In [ ]:
def f(x, y, z):
    return x*y*z + 2*y*z - 3*x*z + 2

print(f"f(0, 0, 0) = {f(0, 0, 0)}")
print(f"f(3, 0, 3) = {f(3, 0, 3)}")
print(f"f(0, 3, 3) = {f(0, 3, 3)}")

## Problem's boundaries

For this example, we will assume that $x$, $y$ and $z$ live in the cube $[0,3]^3$.

In [ ]:
boundaries = [[0,3]]*3

## Observations

We now assume that we make an observation and want to explain it using the model $f$.

We made some measurements and get the value $f(x,y,z)=0$.

However, those measurements aren't exact. There is some noise due to the precision of the sensors we used and even some random quantities at play. Fortunately, we know that the noise can be considered gaussian of mean $0$ and variance $4$.

Our observations sum-up the following way:

$$f(x,y,z) + \nu = 0$$
$$\nu \sim \mathcal{N}(0,\,4)$$

We want to know the probability of $x$ being greater than $2$.

In [ ]:
from random import gauss

sigma = 2

def noisy_f(x, y, z):
    return  f(x, y, z) + gauss(0, sigma)


print(f"f(0, 0, 0) ~ {noisy_f(0, 0, 0)}")
print(f"f(3, 0, 3) ~ {noisy_f(3, 0, 3)}")
print(f"f(0, 3, 3) ~ {noisy_f(0, 3, 3)}")

## Sampler setup

`malcolm-appraiser` needs to communicate with a server that provides the `malcolm-sampler` gRPC service.

Assuming that the server is up on `localhost` and listening on port `7352`, we provide this information to the appraiser with the following.

In [ ]:
import malcolm_appraiser as ma

sampler = ma.MalcolmSampler("localhost:7352")

We then need to register the boundaries of the problem.

In [ ]:
sampler.set_boundaries(boundaries)

The next step is to send a description of the posterior density we want to sample from. In `malcolm-appraiser`, this takes the form of a list of points with density values.

Indeed, the idea here is to approximate the actual posterior density by a staircase density.
We only need to measure the posterior value on set of points.
How to choose which points to measure from is arbitrary and should depend on the task at hand.
Intuitively, we want the set of measurements to contain the most possible information about the actual posterior density.

How to compute the posterior density values for this example is explained in [bayesian_inference.md](./bayesian_inference.md).

Here, we will sample uniformely at random and compute posterior values. This will be sufficient for the example.

In [ ]:
from math import exp
from random import uniform
from statistics import median

N = 1_000

points = []
for _ in range(N):
    points.append(
        [
            uniform(0,3),
            uniform(0,3),
            uniform(0,3),
        ]
    )

posterior_values = []
for x, y, z in points:
    posterior_values.append(exp(-(f(x, y, z)**2)/8))

print("maximal posterior value:", round(max(posterior_values), 2))
print("median posterior value: ", round(median(posterior_values), 2))
print("minimal posterior value:", round(min(posterior_values), 2))

Now we send the information to the sampler.

In [ ]:
sampler.set_posterior(points, posterior_values)

## Sampling and appraisal

Once our tools are set, we just need to sample and compute an average.

For this example, we will compute average by hand.

In [ ]:
from statistics import mean

def h(x, y, z):
    return 1 if x > 2 else 0

probability = mean(map(h, sampler.make_samples(30_000)))

print(f"P( x > 2 | f(x,y,z) + nu = 0) ~= {round(probability, 2)}")

# What comes next?

Et voilà ! Our first example is over. :)

Next, I invite you to have a look at the `malcolm-appraiser` reference, which is not that complicated but provide a little more configuration possibilities than what we used here.

If you have more issues, I invite you to start a thread through the dedicated means so other users can relate.